# Checking PyPSA and Gurobi installation

This is a very simple code to test that PyPSA and Gurobi installation are working properly.

It is based on the example "Demonstrate PyPSA unit commitment with a one-bus two-generator" available as a Jupyter notebook at http://www.pypsa.org/examples/unit-commitment.ipynb.



In [2]:
import pypsa

Initiallize the network, set 4 time steps in the snapshots

In [3]:
network = pypsa.Network()

network.set_snapshots(range(4))

Add one bus named "bus" to the network and add two generators (coal and gas).

Include the data on the nominal power (p_nom), marginal cost (marginal_cost), and minimum power that needs to be dispatched by every generator (p_min_pu). To enable unit commitment on a generator, set its attribute committable = True.

Add the load to the bus.

In [4]:
network.add("Bus","bus")


network.add("Generator","coal",bus="bus",
       committable=True,
       p_min_pu=0.3,
       marginal_cost=20,
       p_nom=10000)

network.add("Generator","gas",bus="bus",
       committable=True,
       marginal_cost=70,
       p_min_pu=0.1,
       p_nom=1000)

network.add("Load","load",bus="bus",p_set=[4000,6000,5000,800])

Solve the linear optimal power flow (lopf) using Gurobi as solver.

In this case, we are optimising the dispatch of every generator to minimise the total system cost.

In [5]:
network.lopf(network.snapshots, solver_name='gurobi')

INFO:pypsa.pf:Slack bus for sub-network 0 is bus
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x21
  Lower bound: 356000.0
  Upper bound: 356000.0
  Number of objectives: 1
  Number of constraints: 25
  Number of variables: 21
  Number of binary variables: 8
  Number of integer variables: 8
  Number of continuous variables: 13
  Number of nonzeros: 45
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality 

('ok', 'optimal')

The result ('ok', 'optimal') indicates that the optimizer has found an optimal solution. Hence, you have checked that it is working. 

Show the status of every generator (if they are working or not)

In [7]:
network.generators_t.status

,coal,gas
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,0.0,1.0


Show the power produced by every generator 

In final hour load goes below part-load limit of coal gen (30%), forcing gas to commit.

In [8]:
network.generators_t.p

,coal,gas
0,4000.0,0.0
1,6000.0,0.0
2,5000.0,0.0
3,0.0,800.0


The total cost can be read from the network objetive.

In [17]:
network.objective

356000.0